# Vbench data_factory框架测试笔记本

这个笔记本提供了一套全面的测试功能，用于验证Vbench框架的各个子模块是否能够正常工作。通过这个笔记本，您可以：

1. 测试数据集加载和处理功能

让我们开始进行测试！

### 图标

## 测试环境设置

首先，我们将设置测试环境，包括必要的目录结构和配置文件

### 工作区，只运行一次

In [1]:
# 导入必要的库
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import yaml
from pprint import pprint

# 获取当前目录
current_dir = os.getcwd()
print(f"当前目录: {current_dir}")

# 设置项目根目录为上一级目录
project_root = os.path.dirname(current_dir)
print(f"项目根目录: {project_root}")
os.chdir(project_root)

# 添加项目根目录到路径
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"✅ 已将项目根目录添加到系统路径: {project_root}")

%load_ext autoreload
%autoreload 2

当前目录: /home/lq/LQcode/2_project/PHMBench/Vbench/test
项目根目录: /home/lq/LQcode/2_project/PHMBench/Vbench
✅ 已将项目根目录添加到系统路径: /home/lq/LQcode/2_project/PHMBench/Vbench


### 导入库

In [2]:
%load_ext autoreload
%autoreload 2

from src.utils.config_utils import load_config, makedir, path_name, transfer_namespace
from src.data_factory import build_data
from src.model_factory import build_model
from src.task_factory import build_task
from src.trainer_factory import build_trainer
from src.utils.metrics_utils import compute_metrics
from src.utils.loss_utils import get_loss_function

print("✅ 成功导入项目模块！")
print("请检查项目结构和安装依赖。")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/lq/.conda/envs/lq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 成功导入项目模块！
请检查项目结构和安装依赖。


### 导入配置文件

In [3]:
config_path='configs/demo/dummy_test.yaml'

print(f"[INFO] 加载配置文件: {config_path}")
configs = load_config(config_path)

# 确保配置中包含必要的部分
required_sections = ['data', 'model', 'task', 'trainer', 'environment']
for section in required_sections:
    if section not in configs:
        print(f"[ERROR] 配置文件中缺少 {section} 部分")


# 设置环境变量和命名空间
args_environment = transfer_namespace(configs.get('environment', {}))

args_data = transfer_namespace(configs.get('data', {}))

args_model = transfer_namespace(configs.get('model', {}).get('args', {}))
args_model.name = configs['model'].get('name', 'default')

args_task = transfer_namespace(configs.get('task', {}).get('args', {}))
args_task.name = configs['task'].get('name', 'default')

args_trainer = transfer_namespace(configs.get('trainer', {}).get('args', {}))
args_trainer.name = configs['trainer'].get('name', 'default')

for key, value in configs['environment'].items():
    if key.isupper():
        os.environ[key] = str(value)
        print(f"[INFO] 设置环境变量: {key}={value}")


[INFO] 加载配置文件: configs/demo/dummy_test.yaml
/home/lq/LQcode/2_project/PHMBench/Vbench
[INFO] 设置环境变量: WANDB_MODE=disabled
[INFO] 设置环境变量: VBENCH_HOME=/home/lq/LQcode/2_project/PHMBench/Vbench
[INFO] 设置环境变量: PYTHONPATH=/home/lq/.conda/envs/lq


### 测试目录

In [4]:
# 创建必要的目录
test_dirs = [
    os.path.join(project_root, "results"),
    os.path.join(project_root, "data/processed"),
    os.path.join(project_root, "data/raw"),
    os.path.join(project_root, "save"),
    os.path.join(project_root, "test/results") 
]

for d in test_dirs:
    os.makedirs(d, exist_ok=True)
    print(f"📁 目录已准备: {d}")

# 设置默认测试配置路径
default_config_path = os.path.join(project_root, "configs/demo/dummy_test.yaml")

# 检查配置文件是否存在
if os.path.exists(default_config_path):
    print(f"✅ 测试配置文件已存在: {default_config_path}")

path, name = path_name(configs, iteration = 1)


📁 目录已准备: /home/lq/LQcode/2_project/PHMBench/Vbench/results
📁 目录已准备: /home/lq/LQcode/2_project/PHMBench/Vbench/data/processed
📁 目录已准备: /home/lq/LQcode/2_project/PHMBench/Vbench/data/raw
📁 目录已准备: /home/lq/LQcode/2_project/PHMBench/Vbench/save
📁 目录已准备: /home/lq/LQcode/2_project/PHMBench/Vbench/test/results
✅ 测试配置文件已存在: /home/lq/LQcode/2_project/PHMBench/Vbench/configs/demo/dummy_test.yaml


## 1. data_factory 数据工厂测试


### data_factory 测试

In [20]:
# 第一次运行构建cache，cache 根据meta_data文件进行命名
data_factory = build_data(args_data,args_task)
# 第二次运行可以直接读取cache
data = data_factory.get_data()
print(f"数据集大小: {len(data)}")
dataset = data_factory.get_dataset()
print(f"数据集大小: {len(dataset)}")
dataloader = data_factory.get_dataloader()
print(f"数据加载器大小: {len(dataloader)}")


所有数据都在缓存中，直接使用缓存文件: /home/lq/LQcode/2_project/PHMBench/Vbench/data/metadata_dummy.h5
data loader length: [1, 1]
max dataloader length: 1 epoch iteration: 2
数据集大小: 2
数据集大小: 2
数据加载器大小: 2


In [7]:
dataloader

### loop dataloader

In [21]:
for i, ((inputs,labels), name) in enumerate(dataloader):
    print(f"第 {i+1} 批数据:")
    print(f"输入: {inputs.shape}")
    print(f"输入: {inputs}")
    print(f"标签: {labels}")
    print(f"名称: {name}")


第 1 批数据:
输入: torch.Size([7, 10, 2])
输入: tensor([[[0.1234, 0.2345],
         [0.2345, 0.3456],
         [0.3456, 0.4567],
         [0.4567, 0.5678],
         [0.5678, 0.6789],
         [0.6789, 0.7890],
         [0.7890, 0.8901],
         [0.8901, 0.9012],
         [0.9012, 0.0123],
         [0.0123, 0.1234]],

        [[0.6789, 0.7890],
         [0.7890, 0.8901],
         [0.8901, 0.9012],
         [0.9012, 0.0123],
         [0.0123, 0.1234],
         [0.1234, 0.2345],
         [0.2345, 0.3456],
         [0.3456, 0.4567],
         [0.4567, 0.5678],
         [0.5678, 0.6789]],

        [[0.6789, 0.7890],
         [0.7890, 0.8901],
         [0.8901, 0.9012],
         [0.9012, 0.0123],
         [0.0123, 0.1234],
         [1.1234, 1.2345],
         [1.2345, 1.3456],
         [1.3456, 1.4567],
         [1.4567, 1.5678],
         [1.5678, 1.6789]],

        [[1.6789, 1.7890],
         [1.7890, 1.8901],
         [1.8901, 1.9012],
         [1.9012, 1.0123],
         [1.0123, 1.1234],
         